In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from graphviz import Digraph
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout, to_agraph
import pygraphviz as pgv

In [10]:
#generating traction graph
edge_count = {}
for k in range(1,51):
    name = 'traction_graph_{}.csv'.format(k)
    cs_matrix = pd.read_csv(name,sep = ",",header=None)
    cs_matrix=np.matrix(np.abs(cs_matrix))
    cs_matrix[:,0]= 0 # make sure no edges to Input
    cs_matrix[1,:]= 0 # clean the edges out from the target
    
    if k == 1: # since in this case Poro is linear with Us, 
            #. we delete Poro to keep algorithm run properly
        labels = {0:"U",1:"T",2:"CN",3:"Fabric"}
    else:
        labels = {0:"U",1:"T",2:"Poro",3:"CN",4:"Fabric"} 
        
    Q= nx.from_numpy_matrix(cs_matrix, create_using = nx.DiGraph())
    edgelists= list(Q.edges())
    sources = []
    targets = []
    edgenames = [] 
    for i in range(len(edgelists)):
        if edgelists[i][0] in labels:
            sources.append(labels[edgelists[i][0]])
        if edgelists[i][1] in labels:
            targets.append(labels[edgelists[i][1]])

    for i in range(len(sources)):
        edgenames.append((sources[i],targets[i]))
        if edgenames[-1] in edge_count:
            edge_count[edgenames[-1]] += 1
        else:
            edge_count[edgenames[-1]] = 0
            
    # if you want to see how each graph works
    # G = nx.DiGraph()
    # G.add_edges_from(edgenames)
    # G.graph['graph']={'rankdir':'TD'}
    # G.graph['node']={'shape':'circle'}
    # G.graph['edges']={'arrowsize':'0.2'}
    # A = to_agraph(G)
    # A.layout('dot')
    # filename = 'elas_graph_{}.png'.format(k)
    # A.draw(filename)

In [2]:
# This is for elas final granula graph
edge_count = {}
for k in range(1,30):
    name = 'final_gns_modified_{}.csv'.format(k)
    cs_matrix = pd.read_csv(name,sep = ",",header=None)
    cs_matrix=np.matrix(np.abs(cs_matrix))
    cs_matrix[:,0]=0
    cs_matrix[1,:]=0

    labels = {0:"Eps",1:"Sig",2:"CN",3:"SFb",4:"degrAssort",
             5:"transty",6:"graphDensity",7:"Average_Clustering",8:"graph_clique_number",9:"local_efficiency",10:"Porosity"}

    Q= nx.from_numpy_matrix(cs_matrix, create_using = nx.DiGraph())
    edgelists= list(Q.edges())
    sources = []
    targets = []
    edgenames = [] 
    for i in range(len(edgelists)):
        if edgelists[i][0] in labels:
            sources.append(labels[edgelists[i][0]])
        if edgelists[i][1] in labels:
            targets.append(labels[edgelists[i][1]])

    for i in range(len(sources)):
        edgenames.append((sources[i],targets[i]))
        if edgenames[-1] in edge_count:
            edge_count[edgenames[-1]] += 1
        else:
            edge_count[edgenames[-1]] = 0
    
    #     G = nx.DiGraph()
    #     G.add_edges_from(edgenames)
    #     G.graph['graph']={'rankdir':'TD'}
    #     G.graph['node']={'shape':'circle'}
    #     G.graph['edges']={'arrowsize':'0.2'}

    #     A = to_agraph(G)
    #     A.layout('dot')
    #     filename = 'elas_gns_{}.png'.format(k)
    #     A.draw(filename)

In [ ]:
from collections import OrderedDict
edge_counted  = OrderedDict(sorted(edge_count.items(), key=lambda t: t[1],reverse = True))
total_cases = 30 # for elas total_cases is 30, for traction problem is 50
for key,value in edge_counted.items():
    edge_counted[key] = (value / total_cases)*100
    
edge_counted

In [14]:
#use a threshold to get the graph
valid_graph_edges = [key for key,value in edge_counted.items() if float(value) > 20]
G = nx.DiGraph()
G.add_edges_from(valid_graph_edges)
G.graph['graph']={'rankdir':'TD'}
G.graph['node']={'shape':'circle'}
G.graph['edges']={'arrowsize':'0.2'}
A = to_agraph(G)
A.layout('dot')
filename = 'trial_final_graph_with_20%.png'
A.draw(filename)

In [13]:
# move the edge in the traction problem
valid_graph_edges.remove(("Fabric","CN"))

In [ ]:
# move the following edges in the elas problem by the rule
valid_graph_edges.remove(('SFb', 'Porosity'))
valid_graph_edges.remove(('SFb', 'degrAssort'))
valid_graph_edges.remove(('local_efficiency', 'Average_Clustering'))
valid_graph_edges.remove(('transty', 'local_efficiency'))
# valid_graph_edges.remove(('graph_clique_number', 'SFb'))
valid_graph_edges.remove(('transty','Average_Clustering'))

In [16]:
for key in valid_graph_edges:
    print(key[0],"->",key[1],"[label = \"{}%\"]".format(int(edge_counted[key])))

Eps -> SFb [label = "93%"]
Eps -> Porosity [label = "93%"]
Eps -> CN [label = "80%"]
CN -> SFb [label = "73%"]
graphDensity -> SFb [label = "73%"]
Average_Clustering -> SFb [label = "73%"]
graph_clique_number -> SFb [label = "73%"]
transty -> SFb [label = "70%"]
local_efficiency -> SFb [label = "70%"]
Average_Clustering -> local_efficiency [label = "66%"]
Eps -> graphDensity [label = "63%"]
local_efficiency -> transty [label = "60%"]
Porosity -> SFb [label = "60%"]
degrAssort -> SFb [label = "53%"]
degrAssort -> transty [label = "46%"]
Eps -> transty [label = "43%"]
Average_Clustering -> transty [label = "36%"]
Eps -> degrAssort [label = "33%"]
Eps -> Average_Clustering [label = "26%"]
CN -> transty [label = "26%"]
CN -> Porosity [label = "23%"]
Eps -> local_efficiency [label = "23%"]
SFb -> Sig [label = "23%"]


In [ ]:
# go to webgraphviz.com and then copy the above to generate the final graph